In [37]:
import os
import json
import pandas
import yaml

# 从yaml文件读取设置
with open('settings.yaml', 'r', encoding='utf-8') as f:
    settings = yaml.safe_load(f)

project_code = settings['project_code']
list_language = settings['list_language']
csv_characters = settings['csv_characters']
csv_focus = settings['csv_focus']
csv_other_loc = settings['csv_other_loc']
dir_output = settings['dir_output']
focus_inlay_window_ID = settings['focus_inlay_window_ID']
focus_inlay_window_x = settings['focus_inlay_window_x']
focus_inlay_window_y = settings['focus_inlay_window_y']
list_seat_variables = settings['list_seat_variables']
list_gui_portrait = settings['list_gui_portrait']
list_gui_name = settings['list_gui_name']
list_gui_trait = settings['list_gui_trait']
leaving_character_variable = settings['leaving_character_variable']
leaving_character_name_loc = settings['leaving_character_name_loc']
leaving_character_trait_loc = settings['leaving_character_trait_loc']
ascending_character_variable = settings['ascending_character_variable']
ascending_character_name_loc = settings['ascending_character_name_loc']
ascending_character_trait_loc = settings['ascending_character_trait_loc']
circle_event_pending_flag = settings['circle_event_pending_flag']
days_recheck_focus = settings['days_recheck_focus']
days_recheck_focus_random_days = settings['days_recheck_focus_random_days']
days_to_start_focus = settings['days_to_start_focus']
days_to_start_focus_random_days = settings['days_to_start_focus_random_days']


In [38]:
# tools in python. just ignore them. 在python中使用的小工具，无视即可

def get_list_depth(lst):
    if isinstance(lst, list):
        return 1 + max(get_list_depth(item) for item in lst)
    else:
        return 0

def read_str(data,null):
    if pandas.isnull(data):
        return null
    else:
        return data

def language_fill(df):
    #language fill
    for i in df.columns:
      if not i in list_language:
         print("wrong language code:{0}, please check\n语言代码：{0}有问题，请检查csv_other_loc".format(i))
    if 'english' in df.columns:
        for i in list_language:
            if not i in df.columns:
                df[i] = df['english']
    else:
        for i in list_language:
            if not i in df.columns:
                df[i] = df.iloc[:,0]
    return df

# 读取并解析JSON数据，如果失败则使用默认值
# Read and parse JSON data, use default values if failed
def load_json(series, column, default, data_name, key):
    try:
        return json.loads(series[column])
    except Exception as e:
        print("{}: Read {} failed. 读取\"{}\"数据失败。（{}）".format(key, column, column, data_name))
        print(e)
        return default

# 创建目录的函数
# Function to create directories
def create_dir(path):
    try:
        os.mkdir(path)
    except:
        pass

# 在钢4中生成的人物工具大全
# Tools you can get for one inputed character in hoi4

对于一个KEY是[character KEY]的人物，你在肛4中可以获得以下工具：

 tools in hoi4 for you to control character who's key is [character KEY]： 

   - trigger:
     
       project_code + "_" + [character KEY] + "_is_on_the_seat"

            检查人物是否在核心圈内有个位置。
            Check if the character has a seat in the inner circle.
          
       project_code + "_" + [character KEY] + "_is_on_the_seat_tt"

          该trigger的tooltips提示
          tooltips for this trigger
          
   - effect:
      
       project_code + "_" + [character KEY] + "_change_trait_effect"

            根据character.csv中给定的trait_limit条件和人物当前状态刷新对应的trait和idea。
            Refresh the corresponding traits and ideas based on the trait_limit conditions given in the character.csv and the current state of the character.

       project_code + "_" + [character KEY] + "_leave_seat"

            让[character KEY]滚。
            let [character KEY] leave his seat.
          
       project_code + "_" + [character KEY] + "_leave_seat_tt"

          [character KEY]离开核心圈时的tooltips提示
          tooltips when [character KEY] leave inner circle


       project_code + "_" + [character KEY] + "_give_seat"

            给[character KEY]一个位置。
            give [character KEY] a seat.
       project_code + "_" + [character KEY] + "_give_seat_tt"

          让[character KEY]加入核心圈的tooltips提示
          tooltips when [character KEY] join inner circle





In [39]:
# create one character
# 生成一个人

def create_one_characters(series):
    key = series['KEY']
    country = series['COUNTRY']
    name = load_json(series, 'NAME', {"english": key}, csv_characters, key)
    desc = load_json(series, 'DESC', {"english": key + '_desc'}, csv_characters, key)
    trait_id = load_json(series, 'TRAIT_id', [key + '_trait_id'], csv_characters, key)
    trait_loc = load_json(series, 'TRAIT_loc', {"english": key + '_trait_loc'}, csv_characters, key)
    l_TRAIT_default = load_json(series, 'TRAIT_default', [key + '_default_trait'], csv_characters, key)

    # 生成角色字符串
    # Generate character string
    characters = """
    {0} = {{
        name = {0}
        portraits = {{
            army = {{
                large = {3}
                small = {4}
            }}
        }}
        advisor = {{
            slot = political_advisor
            idea_token = {0}
            allowed = {{
                original_tag = {1}
            }}
            available = {{
                always = no 
            }}
            traits = {{
                {2}
            }}
            ai_will_do = {{
                factor = 0
            }}
        }}
    }}
    """.format(key, country, '\n'.join(l_TRAIT_default), series['GFX'], series['GFX_small'])
    out_man = characters

    # 角色名字本地化
    # Character name localization
    df_loc_man = pandas.DataFrame({
        key: name,
        key + '_desc': desc,
    }).T
    df_loc_man = language_fill(df_loc_man)

    # 特质名字本地化
    # Trait name localization
    df_loc_trait = pandas.DataFrame(trait_loc).T
    df_loc_trait.columns = trait_id
    df_loc_trait = df_loc_trait.T
    df_loc_trait = language_fill(df_loc_trait)

    # 特质对应的国家精神本地化
    # Idea (according to traits) localization
    df_loc_idea = df_loc_trait.T
    df_loc_idea.columns = list(map(lambda x: x + '_idea', df_loc_idea.columns))
    df_loc_idea = df_loc_idea.T
    df_loc_trait = pandas.concat([df_loc_trait, df_loc_idea])

    return pandas.Series({'key': key, 'out_man': out_man, 'df_loc_man': df_loc_man, 'df_loc_trait': df_loc_trait})

In [40]:
def create_characters(csv_characters, dir_output):
    Df = pandas.read_csv(csv_characters)
    Df_temp = Df.apply(create_one_characters, axis=1)

    # 创建输出目录和子目录
    # Create output directory and subdirectories
    create_dir(dir_output)
    create_dir(os.path.join(dir_output, 'common'))
    create_dir(os.path.join(dir_output, 'common', 'characters'))
    create_dir(os.path.join(dir_output, 'common', 'ideas'))
    create_dir(os.path.join(dir_output, 'common', 'country_leader'))
    create_dir(os.path.join(dir_output, 'localisation'))
    for lang in list_language:
        create_dir(os.path.join(dir_output, 'localisation', lang))

    # 写入characters.txt
    # Write characters.txt
    text = 'characters = {'
    for i in Df_temp.iterrows():
        text += i[1].loc['out_man']
    text += '\n}'
    with open(os.path.join(dir_output, 'common', 'characters', project_code + '_characters.txt'), 'w', encoding='utf_8', newline='\n') as f:
        f.write(text)

    # 写入traits.txt
    # Write traits.txt
    text = 'leader_traits = {'
    for i in Df.iterrows():
        for TRAIT_id, TRAIT_effect in zip(json.loads(i[1].loc["TRAIT_id"]), json.loads(i[1].loc["TRAIT_effect"])):
            text += '''
      {} = {{
         random = no
         {}
      }}
         '''.format(TRAIT_id, TRAIT_effect)
    text += '\n}'
    with open(os.path.join(dir_output, 'common', 'country_leader', project_code + '_traits.txt'), 'w', encoding='utf_8', newline='\n') as f:
        f.write(text)

    # 写入ideas.txt
    # Write ideas.txt
    text = '''ideas = {
         hidden_ideas = {
   '''
    for i in Df.iterrows():
        for TRAIT_id, TRAIT_effect in zip(json.loads(i[1].loc["TRAIT_id"]), json.loads(i[1].loc["TRAIT_effect"])):
            text += '''
      {} = {{
         
            allowed = {{
               always = no
            }}

            removal_cost = -1

            modifier = {{
         {}
         }}
      }}
         '''.format(TRAIT_id + '_idea', TRAIT_effect)
    text += '\n}}'
    with open(os.path.join(dir_output, 'common', 'ideas', project_code + '_ideas.txt'), 'w', encoding='utf_8', newline='\n') as f:
        f.write(text)

    # 写入角色本地化文件
    # Write character localization file
    Df_local_out = pandas.concat(Df_temp.loc[:, 'df_loc_man'].to_list())
    for lang in list_language:
        yml_out = 'l_' + lang + ':\n'
        dict_local = Df_local_out.loc[:, lang].to_dict()
        for k, v in dict_local.items():
            v = v.replace('\n', '\\n')
            yml_out += ' {}: "{}"\n'.format(k, v)
        with open(os.path.join(dir_output, 'localisation', lang, project_code + '_characters_l_' + lang + '.yml'), 'w', encoding='utf_8_sig', newline='\n') as f:
            f.write(yml_out)

    # 写入特质本地化文件
    # Write traits localization file
    Df_local_out = pandas.concat(Df_temp.loc[:, 'df_loc_trait'].to_list())
    for lang in list_language:
        yml_out = 'l_' + lang + ':\n'
        dict_local = Df_local_out.loc[:, lang].to_dict()
        for k, v in dict_local.items():
            v = v.replace('\n', '\\n')
            yml_out += ' {}: "{}"\n'.format(k, v)
        with open(os.path.join(dir_output, 'localisation', lang, project_code + '_traits_l_' + lang + '.yml'), 'w', encoding='utf_8_sig', newline='\n') as f:
            f.write(yml_out)

    return 0

create_characters(csv_characters, dir_output)

0

In [41]:
def characters2inlaywindowfile(csv_characters, list_seat_variables, list_gui_portrait, list_gui_name, list_gui_trait, focus_inlay_window_ID, dir_output):
    Df_characters = pandas.read_csv(csv_characters)

    # 创建输出目录和子目录
    # Create output directory and subdirectories
    create_dir(dir_output)
    create_dir(os.path.join(dir_output, 'common'))
    create_dir(os.path.join(dir_output, 'common', 'scripted_localisation'))
    create_dir(os.path.join(dir_output, 'common', 'focus_inlay_windows'))

    # 写入顾问姓名的scripted_localisation
    # Write name scripted_localisation
    name_text_all = ''

    name_text_one_seat = '''defined_text = {{
    name = {0}'''

    name_text_no_one_on_one_seat = '''
    text = {{ #NO ONE
        trigger = {{
            check_variable = {{ {0} = 0 }}
        }}
        localization_key = empty_key
    }}'''

    name_text_one_character_on_one_seat = '''
    text = {{
        trigger = {{
            check_variable = {{ {0} = {1} }}
        }}
        localization_key = {2}
    }}'''

    for seat, name_id in zip(list_seat_variables + [leaving_character_variable, ascending_character_variable], list_gui_name + [leaving_character_name_loc, ascending_character_name_loc]):
        s_loc_one_seat = name_text_one_seat.format(name_id) + name_text_no_one_on_one_seat.format(seat)
        for _, s_character in Df_characters.iterrows():
            character_id = s_character['ID']
            character_key = s_character['KEY']
            if int(character_id) < 1:
                raise ValueError('character ID can\'t use 0. 人物ID不允许使用0')
            s_loc_one_seat += name_text_one_character_on_one_seat.format(seat, character_id, character_key)
        s_loc_one_seat += '\n}\n'
        name_text_all += s_loc_one_seat

    with open(os.path.join(dir_output, 'common', 'scripted_localisation', project_code + '_seat_name_scripted_loc.txt'), 'w', encoding='utf_8', newline='\n') as f:
        f.write(name_text_all)

    # 写入特质名字的scripted_localisation
    # Write trait scripted_localisation
    trait_text_all = ''
    trait_text_one_character_on_one_seat = '''
    text = {{
        trigger = {{
            check_variable = {{ {0} = {1} }}
            {3}
        }}
        localization_key = {2}
    }}'''
    for seat, gui_trait in zip(list_seat_variables + [leaving_character_variable, ascending_character_variable], list_gui_trait + [leaving_character_trait_loc, ascending_character_trait_loc]):
        s_loc_one_seat = name_text_one_seat.format(gui_trait) + name_text_no_one_on_one_seat.format(seat)
        for _, s_character in Df_characters.iterrows():
            character_id = s_character['ID']
            character_key = s_character['KEY']
            if int(character_id) < 1:
                raise ValueError('character ID should start with 1')
            trait_id = load_json(s_character, 'TRAIT_id', [character_key + '_trait_id'], csv_characters, character_key)
            trait_limit = load_json(s_character, 'TRAIT_limit', [character_key + 'TRAIT_limit'], csv_characters, character_key)
            for limit, id in zip(trait_limit, trait_id):  # Trait names in the higher order in the list will be displayed first. 优先显示列表中顺序靠前的特质名字。
                s_loc_one_seat += trait_text_one_character_on_one_seat.format(seat, character_id, id, limit)
        s_loc_one_seat += '\n}\n'
        trait_text_all += s_loc_one_seat

    with open(os.path.join(dir_output, 'common', 'scripted_localisation', project_code + '_seat_trait_scripted_loc.txt'), 'w', encoding='utf_8', newline='\n') as f:
        f.write(trait_text_all)

    # 写入肖像的scripted_images
    # Write scripted portrait
    text_scripted_portrait = 'scripted_images = {'
    for i, j in zip(list_gui_portrait, list_seat_variables):
        text_one_seat_portrait = '''        {0} = {{
            GFX_blank_leader_portrait = {{
                check_variable = {{ {1} = 0 }}
            }}'''.format(i, j)
        for _, s_character in Df_characters.iterrows():
            character_id = s_character['ID']
            character_gfx = s_character['GFX']
            text_one_seat_portrait += '''
            {0} = {{
                check_variable = {{ {1} = {2} }}
            }}'''.format(character_gfx, j, character_id)
        text_one_seat_portrait += "\n        }\n"
        text_scripted_portrait += text_one_seat_portrait
    text_scripted_portrait += '\n}\n'

    text_inlay_window = '''{0} = {{
            
    window_name = {0}

    internal = yes
    visible = {{
        always=yes
    }}
    {1}
    }}'''.format(focus_inlay_window_ID, text_scripted_portrait)

    with open(os.path.join(dir_output, 'common', 'focus_inlay_windows', project_code + '_seat_name_scripted_loc.txt'), 'w', encoding='utf_8', newline='\n') as f:
        f.write(text_inlay_window)

characters2inlaywindowfile(csv_characters, list_seat_variables, list_gui_portrait, list_gui_name, list_gui_trait, focus_inlay_window_ID, dir_output)

In [42]:
def on_the_seat_trigger_and_change_trait_effect(csv_characters, list_seat_variables, dir_output):
    Df_characters = pandas.read_csv(csv_characters)

    # 创建输出目录和子目录
    # Create output directory and subdirectories
    create_dir(dir_output)
    create_dir(os.path.join(dir_output, 'common'))
    create_dir(os.path.join(dir_output, 'common', 'scripted_effects'))
    create_dir(os.path.join(dir_output, 'common', 'scripted_triggers'))

    # 生成scripted trigger，检查顾问是否在座位上
    # Generate scripted trigger to check if advisor is on the seat
    text_on_the_seat_trigger_all = ''
    text_on_the_seat_check_one_character = '''{0}_is_on_the_seat = {{
    OR = {{
        {1}
    }}
}}
'''

    for s_character in Df_characters.iterrows():
        character_key = s_character[1].loc['KEY']
        character_id = s_character[1].loc['ID']
        all_check = ''.join(["check_variable = {{ {} = {} }}\n".format(seat, character_id) for seat in list_seat_variables])
        text_on_the_seat_trigger_all += text_on_the_seat_check_one_character.format(project_code + "_" + character_key, all_check)

    with open(os.path.join(dir_output, 'common', 'scripted_triggers', project_code + '_characters_on_seat_scripted_triggers.txt'), 'w', encoding='utf_8', newline='\n') as f:
        f.write(text_on_the_seat_trigger_all)

    # 生成scripted effect，在人物状态变化时控制启动国策、改变人物特质与对应的idea，可能还会重置初始国策
    # Generate scripted effect to control starting focus, changing character traits , corresponding ideas, reset first focus when character status changes
    text_change_trait_all = ''
    text_change_trait_one_character = '''{0} = {{
    hidden_effect = {{
'''

    text_remove_trait = '''     if = {{
        limit = {{
            {0} = {{
            has_trait = {1}
            }}
        }}
        remove_ideas = {1}_idea
        remove_trait = {{
            character = {0}
            trait = {1}
            slot = political_advisor
        }}
    }}
'''

    text_change_trait_one_if = '''        if = {{
            limit = {{
                {0}
            }}
            if = {{
                limit = {{
                    {3}
                }}
                add_ideas = {2}_idea
            }}
            add_trait = {{
                character = {1}
                trait = {2}
                slot = political_advisor
            }}
        }}
'''

    text_reset_focus_if = '''        if = {{
            limit = {{
                {0}
            }}
            uncomplete_national_focus = {{
                focus = {1}
                uncomplete_children = no
                refund_political_power = yes
            }}
        }}
'''
    for s_character in Df_characters.iterrows():
        character_key = s_character[1].loc['KEY']
        character_id = s_character[1].loc['ID']
        text_change_trait_all += text_change_trait_one_character.format(project_code + '_' + character_key + '_change_trait_effect')

        # 加载TRAIT_id和TRAIT_limit
        # Load TRAIT_id and TRAIT_limit
        trait_id = load_json(s_character[1], 'TRAIT_id', [character_key + '_trait_id'], csv_characters, character_key)
        trait_limit = load_json(s_character[1], 'TRAIT_limit', [character_key + '_TRAIT_limit'], csv_characters, character_key)

        # 移除所有特质和idea
        # Remove all traits and ideas
        for trait in trait_id:
            text_change_trait_all += text_remove_trait.format(character_key, trait)

        # 添加正确的特质和idea
        # Add correct traits and ideas
        for limit, trait in zip(trait_limit, trait_id):
            text_change_trait_all += text_change_trait_one_if.format(limit, character_key, trait, project_code + "_" + character_key + "_is_on_the_seat = yes")

        # 尝试启动点国策
        # Try to start focus
        text_change_trait_all += '			country_event = {{ id = {}_focus_control.01 }}\n'.format(character_key)

        # 重置初始国策（如果有的话）
        # Reset start_focus(if exists)
        if not pandas.isnull(s_character[1].loc['start_focus']):
            list_start_focus = load_json(s_character[1], 'start_focus', [], csv_characters, character_key)
            for focus in list_start_focus:
                text_change_trait_all += text_reset_focus_if.format(
                    project_code + "_" + character_key + "_is_on_the_seat = no", 
                    focus
                    )

        text_change_trait_all += "\n    }\n}\n"

    with open(os.path.join(dir_output, 'common', 'scripted_effects', project_code + '_characters_change_trait_scripted_effects.txt'), 'w', encoding='utf_8', newline='\n') as f:
        f.write(text_change_trait_all)

on_the_seat_trigger_and_change_trait_effect(csv_characters, list_seat_variables, dir_output)

In [43]:
# leave seat effect, get seat effect, and replace character event

# someone leave seat effect: project_code + "_" + [character KEY] + "_leave_seat"
# 让人下台的scripted_effect: project_code + "_" + [character KEY] + "_leave_seat"

# some one get a seat effect: project_code + "_" + [character KEY] + "_give_seat"
# 让人上台的scripted_effect:project_code + "_" + [character KEY] + "_give_seat"

# all_focus_completed_trigger


def leave_seat_and_give_seat(csv_characters, csv_other_loc, list_seat_variables, dir_output):
    Df_characters = pandas.read_csv(csv_characters)
    Df_other_loc = pandas.read_csv(csv_other_loc)
    Df_other_loc = Df_other_loc.set_index('id')
    Df_other_loc.dropna(subset=['localisation'], inplace=True)

    def jsonloadDf_other_loc(series):
        try:
            name=json.loads(series['localisation'])
        except Exception as e :
            print('Read other_loc_csv row {0} failed. 读取行{0}失败。'.format(str(series.name)))
            print(e)
            name={"english":str(series.name)}
        series['localisation']=name
        return series
    Df_other_loc = Df_other_loc.apply(jsonloadDf_other_loc, axis=1)

    # 创建输出目录和子目录
    # Create output directory and subdirectories
    create_dir(dir_output)
    create_dir(os.path.join(dir_output, 'common'))
    create_dir(os.path.join(dir_output, 'events'))
    create_dir(os.path.join(dir_output, 'localisation'))
    create_dir(os.path.join(dir_output, 'common', 'scripted_effects'))

    # leave_seat_effect
    text_all_leave_seat = ''
    text_one_character_leave_seat = '''{project_code}_{character_KEY}_leave_seat = {{
    custom_effect_tooltip = {{
        localization_key = {project_code}_{character_KEY}_leave_seat_tt
    }}
    hidden_effect = {{
        if = {{
            limit = {{
                {is_on_the_seat_trigger} = yes
            }}
            {text_check_a_seat_leave}
        }}
    }}
}}
'''

    # stop_any_focus and reset_start_focus
    text_stop_any_focus_one_if = '''    else_if = {{
        limit = {{
            has_shine_effect_on_focus = {focus_id}
            check_variable = {{ {who_is_doing_variable} = {character_id} }}
        }}
        deactivate_shine_on_focus = {focus_id}
        clr_country_flag = {in_progress_flag}
        set_variable = {{ {who_is_doing_variable} = 0 }}
    }}
'''

    text_check_a_seat_leave = '''   else_if = {{
        limit = {{
            check_variable = {{ {seat} = {character_id} }}
        }}
        set_variable = {{ {seat} = 0 }}
        {change_trait_effect} = yes
        {text_stop_focus}
    }}
'''



        
    # make one seat empty
    # 清空一个位置

    make_a_seat_empty_one_if = '''      else_if = {{
            limit = {{
                check_variable = {{ {seat} = {character_id} }}
            }}
            {project_code}_{charactre_key}_leave_seat = yes
    }}
'''
    make_all_seat_empty_text=''
    for seat in list_seat_variables:
        make_a_seat_empty_effect = '''{project_code}_make_{seat}_empty = {{
'''.format(project_code=project_code, seat=seat)
        make_a_seat_empty_effect_all_if=''
        for _,s_character in Df_characters.iterrows():
            character_ID = s_character['ID']
            character_KEY = s_character['KEY']

            make_a_seat_empty_effect_all_if += make_a_seat_empty_one_if.format(
            project_code=project_code, 
            seat=seat, 
            character_id=character_ID, 
            charactre_key=character_KEY
            )
        make_a_seat_empty_effect_all_if=make_a_seat_empty_effect_all_if.replace('else_if','if',1)
        make_all_seat_empty_text += make_a_seat_empty_effect+make_a_seat_empty_effect_all_if+'\n}\n'



    # leave seat
    for s_character in Df_characters.iterrows():
        character_KEY = s_character[1].loc['KEY']
        character_ID = s_character[1].loc['ID']

        # text_stop_any_focus_many_if
        complete_focus_order_data = s_character[1].loc['complete_focus_order']
        text_stop_focus = ''
        if type(complete_focus_order_data) == str:
            try:
                list_can_complete_focus = json.loads(complete_focus_order_data)
            except Exception as e:
                print('Read character_csv {0}\'s {1} data failed. 读取character_csv中{0}的{1}数据失败。'.format(str(character_KEY), 'complete_focus_order'))
                raise e
            if type(list_can_complete_focus) == list and len(list_can_complete_focus) > 0:
                text_stop_focus = ''.join([text_stop_any_focus_one_if.format(focus_id=i, character_id=character_ID, in_progress_flag=project_code + "_" + i + "_in_progress_flag", who_is_doing_variable=project_code + "_" + i + "_doing_focus_advisor") for i in list_can_complete_focus])
                text_stop_focus = text_stop_focus.replace('else_if', 'if', 1)
            else:
                print(character_KEY, ': Read complete_focus_order failed, require a list. 读取\"complete_focus_order\"数据失败，请输入一个列表。（csv_character）')

        # text_reset_start_focus
        start_focus = s_character[1].loc['start_focus']
        text_out_reset_start_focus = ''
        if type(start_focus) == str:
            try:
                start_focus = json.loads(start_focus)
            except Exception as e:
                print(character_KEY, ': Read start_focus failed. 读取\"start_focus\"数据失败。（csv_character）')
                print(e)
                start_focus = []
            #if type(start_focus) == list and len(start_focus) > 0:
            #    text_out_reset_start_focus = ''.join([text_reset_start_focus.format(focus_id=i) for i in start_focus])
            #else:
            #    print(character_KEY, ': Read start_focus failed, require a list. 读取\"start_focus\"数据失败，请输入一个列表。（csv_character）')

        # leave_seat_effect and give_seat_effect
        text_all_check_a_seat_leave = ''.join([text_check_a_seat_leave.format(seat=seat, character_id=character_ID, change_trait_effect=project_code + "_" + character_KEY + "_change_trait_effect", 
        text_stop_focus=text_stop_focus) for seat in list_seat_variables])
        text_all_check_a_seat_leave = text_all_check_a_seat_leave.replace('else_if', 'if', 1)

        text_all_leave_seat += text_one_character_leave_seat.format(project_code=project_code, character_KEY=character_KEY, is_on_the_seat_trigger=project_code + "_" + character_KEY + "_is_on_the_seat", text_check_a_seat_leave=text_all_check_a_seat_leave)

    # choose_who_leave_seat_event
    choose_who_leave_seat_event = '''add_namespace = {project_code}_choose_who_leave_seat_event
country_event = {{
    id = {project_code}_choose_who_leave_seat_event.1
    title = {project_code}_choose_who_leave_seat_event_title
    desc = {project_code}_choose_who_leave_seat_event_desc
    picture = "[GetAscendingAdvisorPortrait]"

    is_triggered_only = yes

    immediate = {{
        hidden_effect = {{
            set_country_flag = {circle_event_pending_flag}
        }}
    }}
'''

    choose_who_leave_seat_event_option = '''option = {{ #Replace Ascended Advisor 
    name = {project_code}_choose_who_leave_seat_event.{seat}
    custom_effect_tooltip = {project_code}_choose_who_leave_seat_event_option_tt.{seat}
    ai_chance = {{
        base = 1
    }}

    hidden_effect = {{
        {project_code}_make_{seat}_empty = yes
        set_variable = {{ {seat} = {ascending_character_variable} }}
        {choose_change_trait_effect}
        clear_variable = {ascending_character_variable}
        clear_variable = {leaving_character_variable}
        clr_country_flag = {circle_event_pending_flag}
    }}
}}

'''


    choose_who_leave_seat_event_option_no_one_leave = '''option = {{ #no_one_leave
    name = {project_code}_choose_who_leave_seat_event.no_one_leave
    custom_effect_tooltip = {project_code}_choose_who_leave_seat_event_option_no_one_leave_option_tt
    ai_chance = {{
        base = 1
    }}

    hidden_effect = {{
        {choose_change_trait_effect}
        clear_variable = {ascending_character_variable}
        clear_variable = {leaving_character_variable}
        clr_country_flag = {circle_event_pending_flag}
    }}
}}
'''

    choose_change_trait_effect_one_if = '''           else_if = {{
                limit = {{
                    OR={{ check_variable = {{ {ascending_character_variable} = {character_id} }} check_variable = {{ {leaving_character_variable} = {character_id} }} }}
                }}
                {project_code}_{character_key}_change_trait_effect = yes
            }}
'''

    choose_who_leave_seat_event_out = choose_who_leave_seat_event.format(project_code=project_code, circle_event_pending_flag=circle_event_pending_flag)
    for seat in list_seat_variables:
        choose_change_trait_effect = ''.join([
            choose_change_trait_effect_one_if.format(
                project_code=project_code, 
                character_id=s_character[1].loc['ID'], 
                character_key=s_character[1].loc['KEY'], 
                ascending_character_variable=ascending_character_variable, 
                leaving_character_variable=leaving_character_variable
                ) for s_character in Df_characters.iterrows()
            ])
        choose_change_trait_effect = choose_change_trait_effect.replace('else_if', 'if', 1)
        choose_who_leave_seat_event_out += choose_who_leave_seat_event_option.format(
            project_code=project_code, 
            seat=seat, 
            choose_change_trait_effect=choose_change_trait_effect, 
            ascending_character_variable=ascending_character_variable, 
            leaving_character_variable=leaving_character_variable, 
            circle_event_pending_flag=circle_event_pending_flag
            )
    choose_who_leave_seat_event_out += choose_who_leave_seat_event_option_no_one_leave.format(
        project_code=project_code, 
        choose_change_trait_effect=choose_change_trait_effect, 
        ascending_character_variable=ascending_character_variable, 
        leaving_character_variable=leaving_character_variable, 
        circle_event_pending_flag=circle_event_pending_flag
        )
    choose_who_leave_seat_event_out += '\n}\n'

    # choose_who_leave_seat_event localisation file
    Df_event_loc = pandas.DataFrame(columns=['id', 'loc'])

    # choose_who_leave_seat_event_title
    new_row = pandas.DataFrame([[project_code + '_choose_who_leave_seat_event_title', Df_other_loc.loc['choose_who_leave_seat_event_title', 'localisation']]], columns=Df_event_loc.columns)
    Df_event_loc = Df_event_loc.append(new_row, ignore_index=True)

    # choose_who_leave_seat_event_desc
    new_row = pandas.DataFrame([[project_code + '_choose_who_leave_seat_event_desc', Df_other_loc.loc['choose_who_leave_seat_event_desc', 'localisation']]], columns=Df_event_loc.columns)
    Df_event_loc = Df_event_loc.append(new_row, ignore_index=True)

    # choose_who_leave_seat_event_option_title
    for i, name in zip(list_seat_variables, list_gui_name):
        new_row = pandas.DataFrame([[project_code + '_choose_who_leave_seat_event.' + i, {k: v.format('[{}]'.format(name)) for k, v in Df_other_loc.loc['choose_who_leave_seat_event_option_title', 'localisation'].items()}]], columns=Df_event_loc.columns)
        Df_event_loc = Df_event_loc.append(new_row, ignore_index=True)

    # choose_who_leave_seat_event_option_tt
    for i, name, trait in zip(list_seat_variables, list_gui_name, list_gui_trait):
        new_row = pandas.DataFrame([[project_code + '_choose_who_leave_seat_event_option_tt.' + i, {k: v.format('[{}]'.format(trait), '[{}]'.format(name)) for k, v in Df_other_loc.loc['choose_who_leave_seat_event_option_tt', 'localisation'].items()}]], columns=Df_event_loc.columns)
        Df_event_loc = Df_event_loc.append(new_row, ignore_index=True)

    # choose_who_leave_seat_event_option_no_one_leave_option_title
    new_row = pandas.DataFrame([[project_code + '_choose_who_leave_seat_event.no_one_leave', Df_other_loc.loc['choose_who_leave_seat_event_option_no_one_leave_option_title', 'localisation']]], columns=Df_event_loc.columns)
    Df_event_loc = Df_event_loc.append(new_row, ignore_index=True)

    # choose_who_leave_seat_event_option_no_one_leave_option_tt
    new_row = pandas.DataFrame([[project_code + '_choose_who_leave_seat_event_option_no_one_leave_option_tt', Df_other_loc.loc['choose_who_leave_seat_event_option_no_one_leave_option_tt', 'localisation']]], columns=Df_event_loc.columns)
    Df_event_loc = Df_event_loc.append(new_row, ignore_index=True)

    # tooltips localisation: on_the_seat_tt, leave_seat_tt and give_seat_tt
    Df_loc_seat_tt = pandas.DataFrame()

    for s_character in Df_characters.iterrows():
        # format text
        character_KEY_format = s_character[1].loc['KEY']

        dict_is_on_the_seat_tt = Df_other_loc.loc['is_on_the_seat_tt', 'localisation']
        dict_is_on_the_seat_tt_out = {k: v.format('$' + character_KEY_format + '$') for k, v in dict_is_on_the_seat_tt.items()}

        dict_leave_seat_tt = Df_other_loc.loc['leave_seat_tt', 'localisation']
        dict_leave_seat_tt_out = {k: v.format('$' + character_KEY_format + '$') for k, v in dict_leave_seat_tt.items()}

        dict_give_seat_tt = Df_other_loc.loc['give_seat_tt', 'localisation']
        dict_give_seat_tt_out = {k: v.format('$' + character_KEY_format + '$') for k, v in dict_give_seat_tt.items()}

        # make Df_loc
        Df_loc_seat_tt_temp = pandas.DataFrame({
            project_code + '_' + character_KEY_format + "_is_on_the_seat_tt": dict_is_on_the_seat_tt_out,
            project_code + '_' + character_KEY_format + "_leave_seat_tt": dict_leave_seat_tt_out,
            project_code + '_' + character_KEY_format + "_give_seat_tt": dict_give_seat_tt_out
        })
        Df_loc_seat_tt = pandas.concat([Df_loc_seat_tt, Df_loc_seat_tt_temp.T])

    Df_event_loc_out_all = pandas.DataFrame()
    # write localisation file
    for i in Df_event_loc.iterrows():
        Df_event_loc_out = pandas.DataFrame({
            i[1].loc['id']: i[1].loc['loc'],
        }).T
        Df_event_loc_out_all = pandas.concat([Df_event_loc_out_all, Df_event_loc_out])

    Df_seat_control_loc_out = pandas.concat([Df_event_loc_out_all, Df_loc_seat_tt])
    Df_seat_control_loc_out = language_fill(Df_seat_control_loc_out)

    for i in list_language:
        yml_out = 'l_' + i + ':\n'
        dict_local = Df_seat_control_loc_out.loc[:, i].to_dict()
        for k in dict_local:
            v = dict_local[k].replace('\n', '\\n')
            yml_out += ' {}: "{}"\n'.format(k, v)
        with open(os.path.join(dir_output, 'localisation', i, project_code + '_seat_control_l_' + i + '.yml'), 'w', encoding='utf_8_sig', newline='\n') as f:
            f.write(yml_out)

    # give seat effect
    text_all_give_seat = ''
    text_one_character_give_seat = '''{project_code}_{character_KEY}_give_seat = {{
    custom_effect_tooltip = {{
        localization_key = {project_code}_{character_KEY}_give_seat_tt
    }}
    hidden_effect = {{
        if = {{
            limit = {{
                {project_code}_{character_KEY}_is_on_the_seat = yes
            }}
            log = "this character is already here!"
        }}
'''

    text_check_a_seat_give = '''   else_if = {{
        limit = {{
            check_variable = {{ {seat} = 0 }}
        }}
        set_variable = {{ {seat} = {character_id} }}
        {change_trait_effect} = yes
    }}
'''

    text_no_seat_to_give = '''   else = {{
        set_variable = {{ {ascending_character_variable} = {character_id} }}
        country_event = {project_code}_choose_who_leave_seat_event.1
    }}
'''

    for s_character in Df_characters.iterrows():
        text_one_character_give_seat_temp = text_one_character_give_seat.format(project_code=project_code, character_KEY=s_character[1].loc['KEY'])
        text_one_character_give_seat_temp += ''.join([text_check_a_seat_give.format(seat=seat, character_id=s_character[1].loc['ID'], change_trait_effect=project_code + "_" + s_character[1].loc['KEY'] + "_change_trait_effect") for seat in list_seat_variables])
        text_one_character_give_seat_temp += text_no_seat_to_give.format(project_code=project_code, character_id=s_character[1].loc['ID'], ascending_character_variable=ascending_character_variable)
        text_one_character_give_seat_temp += '\n}\n}\n'
        text_all_give_seat += text_one_character_give_seat_temp

    # write txt
    with open(os.path.join(dir_output, 'events', project_code + '_choose_who_leave_seat_event.txt'), 'w', encoding='utf_8', newline='\n') as f:
        f.write(choose_who_leave_seat_event_out)

    text_effect_out = text_all_leave_seat +'\n'+ make_all_seat_empty_text+ text_all_give_seat 
    with open(os.path.join(dir_output, 'common', 'scripted_effects', project_code + '_characters_in_out_change_scripted_effects.txt'), 'w', encoding='utf_8', newline='\n') as f:
        f.write(text_effect_out)

leave_seat_and_give_seat(csv_characters, csv_other_loc, list_seat_variables, dir_output)


# 自动国策控制
# Auto national focus control

csv_character中，每个character有一个complete_focus_order列表，他们会根据自己的complete_focus_order列表按顺序点国策，如果遇到了未满足开始条件的国策，他们会跳过。如果未来发现先前被跳过的国策的开始条件已经满足，他们会开始先前被跳过的国策。complete_focus_order由国策的ID组成。

csv_focus中的国策间的关系会被读取，并在控制character开始点国策的事件中作为限制条件，控制character不出现同时完成两个互斥国策或是无视国策顺序/条件的情况。

这些关系包括：前提（prerequisite）、互斥（mutually_exclusive）、互斥控制类型（event_control_type）、条件（available）。

另外，circle_event_pending_flag会暂停所有人点国策。


In the csv_character, each character has a list: complete_focus_order, they will complete national focus in order according to this list, and if they encounter a national focus that does not meet the starting conditions, they will skip it. In the future, if they find that the start conditions for a previously skipped focus have been met, they will start the previously skipped focus. complete_focus_order contains national focus ID.

The relationships between focuses in the csv_focus are read and used as a restriction in the event that controls "one character start one focus", so that they will not complete two mutually exclusive focuses at the same time or ignores the order/condition of the focus. 

These relationships conclude: prerequisite, mutually_exclusive, event_control_type, available

Besides，circle_event_pending_flag will pause all characters from starting the national focus。

## 前提关系
## Prerequisite

你在csv_focus的"prerequisite"列里声明了就行。例：

Just declare it in "prerequisite" column in csv_focus. example:

```
    need A:[[a]]
    need A or B:[[a,b]]
    need A and B:[[a],[b]]
    need A and B and C:[[a],[b],[c]]
    need (A or B) and C:[[a,b],[c]]
```

## 互斥关系
## Mutually_exclusive

你需要在csv_focus中的"mutually_exclusive"列声明。需要一个list，包含互斥的全部国策的id

Need to declare it in "mutually_exclusive" cloumn in csv_focus. need a list, contain all mutually exclusive national focuses' ID.

## 互斥控制类型
## event_control_type

互斥控制类型决定了当一个character想要开始某个和其他国策互斥的国策时程序采取什么措施。

本程序提供四种处理互斥国策的类型：事件选择```event_choose```，事件红线```event_redline```，事件提醒```event_remind```，什么也不做```do_nothing```。本程序不会自动生成相关事件的本地化文件。

自动生成的事件id是：```project_code+"_"+event_control_type+"_"+"_".join(list_mutually_exclusive_national_focuses)```

event_control_type control what to do when a character want to start a focus that has mutually exclusive focus. 

This program provides 4 types of policies that deal with mutual exclusions: ```event_choose```, ```event_redline```, ```event_remind```, ```do_nothing```. 

This program will not automatically generate localisation files for these events.

These event have id：```project_code+"_"+event_control_type+"_"+"_".join(list_mutually_exclusive_national_focuses)```

### 事件选择event_choose

仅用于互斥国策。在所有互斥国策中让玩家选一个，封锁其他国策。建议用于两个国策一般会同时满足条件，需要玩家多选一的情况。

Mutually exclusive focuses only. Let the player choose one of all mutually exclusive focuses and block the others. It is recommended to use it when two national focuses generally meet the conditions at the same time and require players to choose one or the other.

### 事件红线event_redline

在一个国策即将开始时，让玩家选择要不要让这个character继续点国策。

When a focus is about to start, let the player choose whether or not to let the character continue to focus on it.

### 事件提醒event_remind

只是跳一个提醒，玩家不能阻止character继续

Just a remind.

### 什么也不做do_nothing

什么也不做。这是默认值。

Do nothing. This is default value.

## 条件
## available
填在这一列的内容会被作为自动生成的国策的开始条件，这个条件也会被原样复制进控制character点国策的事件。

The content in this column will be used as the starting condition of the auto-generated national focus, and this condition will also be copied into the event that controls the character start national focus.



In [44]:
# 生成一个国策
# generate one national focus 

# 让character能够正常点国策只需要填写ID，prerequisite，mutually_exclusive，available四个字段。
# only ID，prerequisite，mutually_exclusive，available columns is necessary for focus auto complete control.

# 一个自动控制的国策在游戏中附带以下工具：
# one auto-controled national focus have these tools in hoi4:

#     country_flag：
#     是否正在被执行in progeress or not：project_code+"_"+focus_id+"_in_progress_flag"

#     variable：
#     谁在执行国策who is doing：project_code+"_"+focus_id+"_doing_focus_advisor"



# 这部分代码需要的本地化数据位于csv_other_loc中，包括：
# The localization data required for this part are located in the csv_other_loc, including:

#     国策正在进行提示器focus_in_progress_tt：project_code+"_inner_circle_focus_in_progress_tt"

#     只有顾问能完成国策提示器focus_can_only_be_completed_by_advisor_tt：
#         不显示名字的提示器no advisor name tt：project_code+"_focus_can_only_be_completed_by_advisor_0_tt"
#         1个顾问能完成的提示器 1 advisor can complete tt：project_code+"_focus_can_only_be_completed_by_advisor_1_tt"
#         2个顾问能完成的提示器 2 advisor can complete tt：project_code+"_focus_can_only_be_completed_by_advisor_2_tt"
#         3个顾问能完成的提示器 3 advisor can complete tt：project_code+"_focus_can_only_be_completed_by_advisor_3_tt"
#         4个顾问能完成的提示器 4 advisor can complete tt：project_code+"_focus_can_only_be_completed_by_advisor_4_tt"



focus_basic="""    focus = {{
        id = {focus_id}
        icon = {icon}

        {text_prerequisite}
        {text_mutually_exclusive}
        {comment_event_control_type}

        x = {x}
        y = {y}
        cost = {cost}
        {relative_position_id}

        available = {{
            {text_available_explain_for_auto_control}
            {available}
            {pause_while_events_are_pending_available}
        }}

        {pause_while_events_are_pending_other_code}

        completion_reward = {{
            {completion_reward}
        }}

    }}

"""

available_explain_for_auto_control='''if = {{ #ADVISOR WORKING ON THE FOCUS (Show progress)
                limit = {{
                    has_country_flag = {focus_in_progress_flag}
                }}
                set_temp_variable = {{ temp_focus_days = {focus_cost} }} #For tooltip purposes
                multiply_temp_variable = {{ temp_focus_days = 7 }} #The constant uses focus cost which is in weeks, so convert to days
                
                {text_available_for_auto_control_someone_is_working}
            }}

            {text_available_for_auto_control_no_one_is_working}
'''# focus_in_progress_flag, focus_cost, text_available_for_auto_control_someone_is_working, text_available_for_auto_control_no_one_is_working

available_for_auto_control_one_character_is_working='''                else_if = {{ #someone working on it
                    limit = {{
                        check_variable = {{ {doing_focus_advisor} = {character_id} }}
                    }}
                    custom_override_tooltip = {{
                        tooltip = {{
                            localization_key = {inner_circle_focus_in_progress_tt}
                            CHARACTER = {character_key}
                            FLAG_DAYS = [?{focus_in_progress_flag}:days]
                        }}
                        always = no 
                    }}
                }}
'''# focus_in_progress_flag, focus_cost, doing_focus_advisor(variable name), character_id, character_key, inner_circle_focus_in_progress_tt

available_for_auto_control_no_one_is_working='''else = {{ #NOT BEEN WORKED ON (Just Explain)
                custom_override_tooltip = {{
                    tooltip = {{
                        localization_key = {focus_can_only_be_completed_by_advisor_tt}
{many_characters}
                    }}
                    always = no
                }}
            }}
'''# focus_can_only_be_completed_by_advisor_tt, many_characters 

# one_character + one_character + ...... = many_characters
one_character='''                        CHARACTER{num} = {character_key}
'''# num, character_key, focus_id

def genarate_one_focus(s_focus,Df_characters):

    status_code=0
    focus_all_text=''

    #ID
    # ID
    focus_id=read_str(s_focus['ID'],'')

    #icon
    # 图标
    icon=read_str(s_focus['icon'],'')

    # text_prerequisite
    # 前提条件文本
    data_prerequisite = s_focus['prerequisite']
    text_prerequisite = ''
    if not pandas.isnull(data_prerequisite):
        list_prerequisite = load_json(s_focus, 'prerequisite', '', "csv_focus", focus_id) 
        if type(list_prerequisite) != list:
            print('Read focus_csv row {0} columns {1}failed, require list, got wrong data type. 读取focus_csv行{0}列{1}失败。这里应该输入一个list'.format(str(s_focus['ID']), 'prerequisite'))
            s_focus['focus_all_text'] = focus_all_text
            return focus_all_text, status_code
        if get_list_depth(list_prerequisite) != 2:
            print('Read focus_csv row {0} columns {1}failed, require depth = 2 list, got list with wrong depth. 读取focus_csv行{0}列{1}失败。这里应该输入一个2层的list'.format(str(s_focus['ID']), 'prerequisite'))
            s_focus['focus_all_text'] = focus_all_text
            return focus_all_text, status_code

        for row in list_prerequisite:
            text_row = ''
            for focus in row:
                text_row += ' focus = ' + focus
            text_prerequisite += 'prerequisite = {' + text_row + ' }\n        '

    # text_mutually_exclusive
    # 互斥条件文本
    data_mutually_exclusive = s_focus['mutually_exclusive']
    text_mutually_exclusive=''
    if not pandas.isnull(data_mutually_exclusive):
        list_mutually_exclusive=load_json(s_focus, 'mutually_exclusive', '', "csv_focus", focus_id) 

        if type(list_mutually_exclusive)!=list:
            # error: wrong data type
            # 错误：数据类型错误
            print('Read focus_csv row {0} columns {1}failed, require list, got wrong data type. 读取focus_csv行{0}列{1}失败。这里应该输入一个list'.format(str(s_focus['ID'],'mutually_exclusive')))
            s_focus['focus_all_text']=focus_all_text
            return focus_all_text, status_code
        if get_list_depth(list_mutually_exclusive)!=1:
            # error: wrong depth
            # 错误：列表深度错误
            print('Read focus_csv row {0} columns {1}failed, require depth = 1 list, got list with wrong depth. 读取focus_csv行{0}列{1}失败。这里应该输入一个1层的list'.format(str(s_focus['ID'],'mutually_exclusive')))
            s_focus['focus_all_text']=focus_all_text
            return focus_all_text, status_code

        text_row=''
        for focus in list_mutually_exclusive:
            text_row+=' focus = '+focus
        text_mutually_exclusive+='mutually_exclusive = {' + text_row+' }\n        '

    # comment_event_control_type
    # 事件控制类型注释
    data_event_control_type = s_focus['event_control_type']
    if pandas.isnull(data_event_control_type):
        comment_event_control_type=''
    else:
        comment_event_control_type='# event_control_type: '+str(data_event_control_type)
    
    # x
    # x 坐标
    x=s_focus['x']
    if not type(x)==int:
        print(focus_id+'的x值有问题。','\"x\" data in '+focus_id+'has something wrong')

    # y
    # y 坐标
    y=s_focus['y']
    if not type(y)==int:
        print(focus_id+'的y值有问题。','\"y\" data in '+focus_id+'has something wrong')

    # cost
    # 花费时间
    cost=s_focus['cost']
    if not type(cost)==int:
        print(focus_id+'的cost值有问题。','\"cost\" data in '+focus_id+'has something wrong')
    
    # relative_position_id
    # 位置坐标是相对于哪个国策
    data_relative_position_id = s_focus['relative_position_id']
    if pandas.isnull(data_relative_position_id):
        relative_position_id=''
    else:
        relative_position_id='relative_position_id = '+str(data_relative_position_id)

    # variables and country flags
    # 变量和国家标志
    focus_in_progress_flag = project_code+"_"+focus_id+"_in_progress_flag"
    doing_focus_advisor = project_code+"_"+focus_id+"_doing_focus_advisor"
    inner_circle_focus_in_progress_tt=project_code+"_inner_circle_focus_in_progress_tt"

    # list of character who can complete this focus
    # 能完成这个国策的顾问的列表

    list_character_KEY_can_do=[]
    list_character_ID_can_do=[]

    for row in Df_characters.iterrows():
        character_KEY = row[1].loc['KEY']
        character_ID = row[1].loc['ID']
        complete_focus_order_data = row[1].loc['complete_focus_order']
        try:
            list_can_complete_focus=json.loads(complete_focus_order_data)
        except Exception as e :
            print('Read character_csv {0}\'s {1} data failed. 读取character_csv中{0}的{1}数据失败。'.format(str(character_KEY,'complete_focus_order')))
            print()
            raise e
        if not pandas.isnull(complete_focus_order_data):
            if focus_id in list_can_complete_focus:
                list_character_KEY_can_do += [character_KEY]
                list_character_ID_can_do += [character_ID]

    # text_available_explain_for_auto_control
    text_available_explain_for_auto_control=''


    # 如果有顾问能完成这个国策
    # If there are advisors who can complete this focus
    if len(list_character_ID_can_do)>0:

        #text_available_for_auto_control_someone_is_working
        text_available_for_auto_control_someone_is_working=''

        for character_id,character_key in zip(list_character_ID_can_do,list_character_KEY_can_do):
            #fill text_available_for_auto_control_someone_is_working
            text_available_for_auto_control_someone_is_working+=available_for_auto_control_one_character_is_working.format(
                doing_focus_advisor=doing_focus_advisor,
                character_id=character_id,
                character_key=character_key,
                inner_circle_focus_in_progress_tt=inner_circle_focus_in_progress_tt,
                focus_in_progress_flag=focus_in_progress_flag
                )
        text_available_for_auto_control_someone_is_working=text_available_for_auto_control_someone_is_working.replace('                else_if','if',1)
        
        #text_available_for_auto_control_no_one_is_working

            #choose focus_can_only_be_completed_by_advisor_tt according to the num of character can complete this focus
        if len(list_character_KEY_can_do)>4:
            focus_can_only_be_completed_by_advisor_tt=project_code+"_focus_can_only_be_completed_by_advisor_0_tt"
        else:
            focus_can_only_be_completed_by_advisor_tt=project_code+"_focus_can_only_be_completed_by_advisor_"+str(len(list_character_KEY_can_do))+"_tt"
        
        many_characters=''
        for character_key,num in zip(list_character_KEY_can_do,range(1,len(list_character_KEY_can_do)+1)):
            #fill many_characters with one_character
            many_characters+=one_character.format(
                num=num,
                character_key=character_key,
                focus_id=focus_id
            )

        text_available_for_auto_control_no_one_is_working=available_for_auto_control_no_one_is_working.format(
            focus_can_only_be_completed_by_advisor_tt=focus_can_only_be_completed_by_advisor_tt,
            many_characters=many_characters
            )

        text_available_explain_for_auto_control=available_explain_for_auto_control.format(
            focus_in_progress_flag=focus_in_progress_flag,
             focus_cost=cost, text_available_for_auto_control_someone_is_working=text_available_for_auto_control_someone_is_working,
             text_available_for_auto_control_no_one_is_working=text_available_for_auto_control_no_one_is_working
             )
    else:
        pass

    # available
    available=read_str(s_focus['available'],'')

    # pause_while_events_are_pending
    # 事件待定时是否要暂停
    pause_while_events_are_pending=False
    if not pandas.isnull(s_focus['pause_while_events_are_pending']):
        if s_focus['pause_while_events_are_pending']:
            pause_while_events_are_pending=True
    
    # if pause_while_events_are_pending is true, these code will be added in available part
    # 如果事件待定时要暂停，补充以下代码到available部分
    if pause_while_events_are_pending:
        pause_while_events_are_pending_available='''			if = {{
				limit = {{
					has_country_flag = {0}
				}}
				NOT = {{ has_country_flag = {0} }}
			}}'''.format(circle_event_pending_flag)
    else:
        pause_while_events_are_pending_available=''

    # if pause_while_events_are_pending is true, these code will be added.
    # 如果事件待定时要暂停，补充以下代码到
    if pause_while_events_are_pending:
        pause_while_events_are_pending_other_code='''		cancel_if_invalid = no
		continue_if_invalid = no'''
    else:
        pause_while_events_are_pending_other_code=''

    # completion_reward
    # 完成奖励
    completion_reward=read_str(s_focus['completion_reward'],'')

    # Generate the whole focus in P language code.
    # 生成整个国策的P语言代码
    focus_all_text=focus_basic.format(
        focus_id=focus_id,
        icon=icon,
        text_prerequisite=text_prerequisite,
        text_mutually_exclusive=text_mutually_exclusive,
        comment_event_control_type=comment_event_control_type,
        x=x,
        y=y,
        cost=cost,
        relative_position_id=relative_position_id,
        text_available_explain_for_auto_control=text_available_explain_for_auto_control,
        available=available,
        pause_while_events_are_pending_available=pause_while_events_are_pending_available,
        pause_while_events_are_pending_other_code=pause_while_events_are_pending_other_code,
        completion_reward=completion_reward

    )

    status_code=1


    return focus_all_text, status_code


In [45]:
# 全部国策批量生成
# generate all national focus 

def focus_generate_all(csv_characters,csv_focus,csv_other_loc,focus_tree_id, focus_tree_tag,focus_inlay_window_ID, focus_inlay_window_x,focus_inlay_window_y,dir_output):
    # 读取CSV文件
    # Read CSV files
    Df_characters = pandas.read_csv(csv_characters)
    Df_focus = pandas.read_csv(csv_focus)
    Df_other_loc = pandas.read_csv(csv_other_loc)

    # 设置索引并删除缺少本地化数据的行
    # Set index and drop rows with missing localization data
    Df_other_loc = Df_other_loc.set_index('id')
    Df_other_loc.dropna(subset=['localisation'], inplace=True)

    # 创建输出目录
    # Create output directories
    for subdir in ['common/national_focus', 'localisation']:
        os.makedirs(os.path.join(dir_output, subdir), exist_ok=True)
    
    # 遍历每个国策并生成文本
    # Iterate through each focus and generate text
    all_focus=''

    for index, s_focus in Df_focus.iterrows():
        one_focus_text, status = genarate_one_focus(s_focus, Df_characters)
        if status != 1:
            print("国策{}生成错误，已跳过".format(s_focus['ID']), "National focus {} has error, skipped".format(s_focus['ID']))
        else:
            all_focus += one_focus_text


    # 生成国策树的文本
    # Generate focus tree text
    focus_tree = '''focus_tree = {{
    id = {focus_tree_id}
    
	country = {{
		factor = 0
		
		modifier = {{
			add = 10
			tag = {focus_tree_tag}
		}}
	}}

	initial_show_position = {{
		x = 76
		y = 0
	}}
    
    default = no
    
    inlay_window = {{
        id = {focus_inlay_window_ID}
        position = {{ x = {focus_inlay_window_x} y = {focus_inlay_window_y} }}
    }}
    
    {all_focus}
}}'''.format(
        focus_tree_id=focus_tree_id,
        focus_tree_tag=focus_tree_tag,
        focus_inlay_window_ID=focus_inlay_window_ID,
        focus_inlay_window_x=focus_inlay_window_x,
        focus_inlay_window_y=focus_inlay_window_y,
        all_focus=all_focus
    )

    # 将国策树文本写入文件
    # Write focus tree text to file
    with open(os.path.join(dir_output, 'common/national_focus', '{}_{}_auto.txt'.format(project_code,focus_tree_id)), 'w', encoding='utf-8') as f:
        f.write(focus_tree)

    # 本地化文件
    # localisation_file

	# 国策本地化
    # focus localisation
    dict_loc_focus={}
    for _ , i in Df_focus.iterrows():
        focus_id=i['ID']
        dict_loc = load_json(i, 'loc', {'english': focus_id}, 'csv_focus', focus_id)
        dict_loc_desc = load_json(i, 'desc', {'english': focus_id + '_desc'}, 'csv_focus', focus_id)
        dict_loc_focus[focus_id] = dict_loc
        dict_loc_focus[focus_id + '_desc'] = dict_loc_desc
    Df_loc_focus=pandas.DataFrame(dict_loc_focus)
    Df_loc_focus=Df_loc_focus.T

    Df_loc_focus =language_fill(Df_loc_focus)

    # 国策只能由顾问完成提示
    # focus can only be completed by advisor tooltips

    Df_loc_focus_other = pandas.DataFrame(
        {
            project_code + "_inner_circle_focus_in_progress_tt": load_json(Df_other_loc.loc['inner_circle_focus_in_progress_tt', :], 'localisation', {"english":'inner_circle_focus_in_progress_tt'}, 'Df_other_loc', 'inner_circle_focus_in_progress_tt'),
            project_code + "_focus_can_only_be_completed_by_advisor_0_tt": load_json(Df_other_loc.loc['focus_can_only_be_completed_by_advisor_0_tt', :], 'localisation', {"english":'focus_can_only_be_completed_by_advisor_0_tt'}, 'Df_other_loc', 'focus_can_only_be_completed_by_advisor_0_tt'),
            project_code + "_focus_can_only_be_completed_by_advisor_1_tt": load_json(Df_other_loc.loc['focus_can_only_be_completed_by_advisor_1_tt', :], 'localisation', {"english":'focus_can_only_be_completed_by_advisor_1_tt'}, 'Df_other_loc', 'focus_can_only_be_completed_by_advisor_1_tt'),
            project_code + "_focus_can_only_be_completed_by_advisor_2_tt": load_json(Df_other_loc.loc['focus_can_only_be_completed_by_advisor_2_tt', :], 'localisation', {"english":'focus_can_only_be_completed_by_advisor_2_tt'}, 'Df_other_loc', 'focus_can_only_be_completed_by_advisor_2_tt'),
            project_code + "_focus_can_only_be_completed_by_advisor_3_tt": load_json(Df_other_loc.loc['focus_can_only_be_completed_by_advisor_3_tt', :], 'localisation', {"english":'focus_can_only_be_completed_by_advisor_3_tt'}, 'Df_other_loc', 'focus_can_only_be_completed_by_advisor_3_tt'),
            project_code + "_focus_can_only_be_completed_by_advisor_4_tt": load_json(Df_other_loc.loc['focus_can_only_be_completed_by_advisor_4_tt', :], 'localisation', {"english":'focus_can_only_be_completed_by_advisor_4_tt'}, 'Df_other_loc', 'focus_can_only_be_completed_by_advisor_4_tt')
        }
    )
    Df_loc_focus_other=Df_loc_focus_other.T

    # language fill
    Df_loc_focus_other=language_fill(Df_loc_focus_other)


    Df_loc_out=pandas.concat([Df_loc_focus,Df_loc_focus_other])
    # write loc file
    for i in list_language:
        yml_out='l_'+i+':\n'
        dict_local=Df_loc_out.loc[:,i].to_dict()
        for k in dict_local:
            v=dict_local[k].replace('\n','\\n')
            yml_out+=' {}: "{}"\n'.format(k,v)
        with open(os.path.join(dir_output,'localisation',i,project_code+'_auto_controled_focuses_l_'+i+'.yml'),'w',encoding='utf_8_sig', newline='\n')as f:
            f.write(yml_out)


focus_generate_all(csv_characters,csv_focus,csv_other_loc,'tan_focus', 'TAN', focus_inlay_window_ID, focus_inlay_window_x,focus_inlay_window_y,dir_output)


In [46]:
# 一个character完成全部国策trigger
# trigger checks if one character completed all his focus

def all_focus_completed_trigger(csv_characters, csv_focus, dir_output):
    Df_characters = pandas.read_csv(csv_characters)
    Df_focus = pandas.read_csv(csv_focus)

    # 创建输出目录和子目录
    # Create output directory and subdirectories
    create_dir(dir_output)
    create_dir(os.path.join(dir_output, 'common'))
    create_dir(os.path.join(dir_output, 'common', 'scripted_triggers'))

    # 生成scripted trigger，检查某个character的所有国策是否完成
    # Generate scripted trigger to check if one character's all focuses are completed
    text_all_focus_completed_trigger = ''
    text_one_character_all_focus_completed = '''{project_code}_{character_KEY}_all_focus_completed = {{
    AND = {{
        {all_focus_check}
    }}
}}
'''

    for s_character in Df_characters.iterrows():
        character_KEY = s_character[1].loc['KEY']
        complete_focus_order_data = s_character[1].loc['complete_focus_order']
        all_focus_check = ''

        if type(complete_focus_order_data) == str:
            try:
                list_can_complete_focus = json.loads(complete_focus_order_data)
            except Exception as e:
                print('Read character_csv {0}\'s {1} data failed. 读取character_csv中{0}的{1}数据失败。'.format(str(character_KEY), 'complete_focus_order'))
                raise e

            if type(list_can_complete_focus) == list and len(list_can_complete_focus) > 0:
                all_focus_check = ''.join(['has_completed_focus = {}\n'.format(focus_id) for focus_id in list_can_complete_focus])
            else:
                print(character_KEY, ': Read complete_focus_order failed, require a list. 读取"complete_focus_order"数据失败，请输入一个列表。（csv_character）')

        text_all_focus_completed_trigger += text_one_character_all_focus_completed.format(
            project_code=project_code,
            character_KEY=character_KEY,
            all_focus_check=all_focus_check
        )

    with open(os.path.join(dir_output, 'common', 'scripted_triggers', project_code + "_characters_completed_all_focus_scripted_triggers.txt"), 'w', encoding='utf_8', newline='\n') as f:
        f.write(text_all_focus_completed_trigger)

all_focus_completed_trigger(csv_characters, csv_focus, dir_output)

In [47]:
# focus auto control event line
# 国策自动控制事件链

# namespace
add_namespace='add_namespace = {character_key}_focus_control\n'



# control characters start focus 控制人物开始国策
start_event = '''country_event = {{# start focus event
	id = {character_key}_focus_control.01
	
	hidden = yes

	is_triggered_only = yes

	trigger = {{
		{is_on_the_seat_trigger} = yes
		NOT = {{has_country_flag = {has_start_focus_control_event_flag}}}
	}}

	immediate = {{
		set_country_flag = {has_start_focus_control_event_flag}
		{many_focus_many_if}
		{all_completed_else}
		{no_focus_else}
	}}
}}
'''# character_key,focus_in_progress_flag,is_on_the_seat_trigger,many_focus_many_if,all_completed_else,no_focus_else,has_start_focus_control_event_flag



# has event_control_type one if
one_focus_start_one_if_event_control_type='''		else_if = {{
			limit = {{
{limit}
				NOT = {{ has_shine_effect_on_focus = {focus_id}}}
				NOT = {{ has_completed_focus = {focus_id} }}
                NOT = {{ has_country_flag = {project_code}_{focus_id}_waiting }}
			}}
			country_event = {{ id = {event_id}}}

		}}
'''# limit,character_key,character_id,focus_id,in_progress_flag,costX7_add_1,who_is_doing_variable




# event_choose
one_focus_event_choose_event_namespace='{project_code}_{character_key}_choose.{focus_id}'
one_focus_event_choose_event='''country_event = {{
	id = {project_code}_{character_key}_choose.{focus_id}.01

	is_triggered_only = yes

	trigger = {{
		{is_on_the_seat_trigger} = yes
	}}

	immediate = {{
		set_country_flag = {circle_event_pending_flag}
        {set_waiting_flags}
	}}
	{options}
}}
'''#character_key, focus_id, postion_num_of_focus, is_on_the_seat_trigger, circle_event_pending_flag,pause_while_events_are_pending_other_code


choose_focus_event_option = '''option = {{
	name = {project_code}_{character_key}_choose.{focus_id}.{choose_focus_id}
	custom_effect_tooltip = {project_code}_{character_key}_choose.{focus_id}.{choose_focus_id}_tt
	ai_chance = {{
		base = 1
	}}
	trigger = {{
		{limit}
	}}
	hidden_effect = {{
		activate_shine_on_focus = {choose_focus_id}
		set_country_flag = {{ flag = {in_progress_flag} days = {costX7_add_1} value = 1 }}
		set_variable = {{ {who_is_doing_variable} = {character_id} }}
		country_event = {{ id = {character_key}_focus_control.02 days = {costX7_add_1} }}
        clr_country_flag = {circle_event_pending_flag}
		{clr_waiting_flags}
        
	}}
}}
'''


# event_redline
one_focus_event_redline_event_namespace='{project_code}_{character_key}_redline.{focus_id}'
one_focus_event_redline_event='''country_event = {{
	id = {project_code}_{character_key}_redline.{focus_id}.01

	is_triggered_only = yes

	trigger = {{
		{is_on_the_seat_trigger} = yes
	}}

	immediate = {{
		set_country_flag = {circle_event_pending_flag}
		set_country_flag = {project_code}_{focus_id}_waiting
	}}
	{options}
}}
'''#character_key, focus_id, postion_num_of_focus, is_on_the_seat_trigger, circle_event_pending_flag,pause_while_events_are_pending_other_code

redline_event_start_focus_option = '''	option = {{ 
		name = {project_code}_{character_key}_redline.{focus_id}.continue
		custom_effect_tooltip = {project_code}_{character_key}_redline.{focus_id}.continue_tt
		ai_chance = {{
			base = 1
		}}
		hidden_effect = {{
			activate_shine_on_focus = {focus_id}
			set_country_flag = {{ flag = {in_progress_flag} days = {costX7_add_1} value = 1 }}
			set_variable = {{ {who_is_doing_variable} = {character_id} }}
			clr_country_flag = {circle_event_pending_flag}
            clr_country_flag = {project_code}_{focus_id}_waiting
			country_event = {{ id = {character_key}_focus_control.02 days = {costX7_add_1} }}

		}}
	}}
'''
redline_event_stop_focus_option = '''	option = {{
		name = {project_code}_{character_key}_redline.{focus_id}.stop
		custom_effect_tooltip = {project_code}_{character_key}_redline.{focus_id}.stop_tt
		ai_chance = {{
			base = 1
		}}
		hidden_effect = {{
			set_country_flag = {focus_id}_was_stopped
            clr_country_flag = {circle_event_pending_flag}
            clr_country_flag = {project_code}_{focus_id}_waiting
			country_event = {{ id = {character_key}_focus_control.01 days = {days_to_start_focus} random_days = {days_to_start_focus_random_days} }}
		}}
	}}
'''


# event_remind
one_focus_event_remind_event_namespace='{project_code}_{character_key}_remind'
one_focus_event_remind_event='''country_event = {{
	id = {project_code}_{character_key}_remind.{focus_id}

	is_triggered_only = yes

	trigger = {{
		{is_on_the_seat_trigger} = yes
	}}

	immediate = {{
		activate_shine_on_focus = {focus_id}
		set_country_flag = {{ flag = {in_progress_flag} days = {costX7_add_1} value = 1 }}
		set_variable = {{ {who_is_doing_variable} = {character_id} }}
		country_event = {{ id = {character_key}_focus_control.02 days = {costX7_add_1} }}
	}}
	option = {{
		name = {project_code}_{character_key}_remind.{focus_id}.ok
		custom_effect_tooltip = {project_code}_{character_key}_remind.{focus_id}.ok_tt
		ai_chance = {{
			base = 1
		}}

	}}
}}
'''#character_key, focus_id, postion_num_of_focus, is_on_the_seat_trigger, circle_event_pending_flag,pause_while_events_are_pending_other_code


# no_control

one_focus_start_one_if_no_control='''		else_if = {{
			limit = {{
{limit}
				NOT = {{ has_shine_effect_on_focus = {focus_id}}}
				NOT = {{ has_completed_focus = {focus_id} }}
                NOT = {{ has_country_flag = {project_code}_{focus_id}_waiting }}
			}}
			activate_shine_on_focus = {focus_id}
			set_country_flag = {{ flag = {in_progress_flag} days = {costX7_add_1} value = 1 }}
			set_variable = {{ {who_is_doing_variable} = {character_id} }}
			country_event = {{ id = {character_key}_focus_control.02 days = {costX7_add_1} }}

		}}
'''# limit,character_key,character_id,focus_id,in_progress_flag,costX7_add_1,who_is_doing_variable


all_completed_else='''		else_if = {{
			limit = {{
				{all_focus_completed_trigger} = yes
			}}
			#log = "{character_key} completed all his focus"
		}}

'''

one_focus_start_no_focus_else='''		else = {{ #Debug logs
			#log = "SOMETHING WENT WRONG WITH {character_key}'s FOCUS BRANCH - Cannot start new focus"
			set_country_flag = {has_start_focus_control_event_flag}
			country_event = {{ id = {character_key}_focus_control.01 days = {days_to_start_focus} random_days = {days_to_start_focus_random_days} }}
		}}
'''

# control characters complete focus 控制人物完成国策
complete_event = '''country_event = {{
	id = {character_key}_focus_control.02

	hidden = yes

	is_triggered_only = yes

	trigger = {{
		{is_on_the_seat_trigger} = yes
	}}

	immediate = {{
		{many_focus_many_if}
		{no_focus_else}
	}}
}}
'''

one_focus_complete_one_if='''		else_if = {{
			limit = {{
				has_shine_effect_on_focus = {focus_id}
				check_variable = {{ {who_is_doing_variable} = {character_id} }} 
				{limit}
			}}
			complete_national_focus = {{
				focus = {focus_id}
				use_side_message = yes
				originator_name = {character_key}
			}}
			clr_country_flag = {in_progress_flag}
			clr_country_flag = {has_start_focus_control_event_flag}
			country_event = {{ id = {character_key}_focus_control.01 days = {days_to_start_focus} random_days = {days_to_start_focus_random_days} }}
		}}
'''

one_focus_complete_one_limit=''
one_focus_complete_no_focus_else='''		else = {{ #Debug logs
			#log = "SOMETHING WENT WRONG WITH {character_key}'s FOCUS BRANCH - Cannot complete focus"
			clr_country_flag = {in_progress_flag}
			clr_country_flag = {has_start_focus_control_event_flag}
			country_event = {{ id = {character_key}_focus_control.01 days = {days_to_start_focus} random_days = {days_to_start_focus_random_days} }}

		}}
'''


In [48]:

# start_focus_event
def start_focus_event(csv_characters,csv_focus,dir_output):
	Df_characters=pandas.read_csv(csv_characters)
	Df_focus=pandas.read_csv(csv_focus)
	#mkdir
	try:
		os.mkdir(dir_output)
	except:
		pass
	try:
		os.mkdir(os.path.join(dir_output,'events'))
	except:
		pass
	# event control all focus start
	start_focus_event_all_namespace=''
	start_focus_event_all_events=''
	# event control focus start event_control_type
	event_control_type_event_namespace_all=''
	event_control_type_event_all=''

	for row in Df_characters.iterrows():
		s_character=row[1]
		character_KEY=s_character['KEY']
		character_id=s_character['ID']
		complete_focus_order_data = s_character['complete_focus_order']

		# namespace
		namespace=add_namespace.format(character_key=character_KEY)
		start_focus_event_all_namespace+=namespace
		# event control focus start event_control_type
		event_control_type_event_namespace_one_character=[]
		event_control_type_event_one_character=''
		# focus control events
		if type(complete_focus_order_data)==str:
			try:
				list_can_complete_focus=json.loads(complete_focus_order_data)
			except Exception as e :
				print('Read character_csv {0}\'s {1} data failed. 读取character_csv中{0}的{1}数据失败。'.format(str(character_KEY,'complete_focus_order')))
				print()
				raise e

			if not type(list_can_complete_focus)==list:
				print(character_KEY,': Read complete_focus_order failed, require a list. 读取"complete_focus_order"数据失败，请输入一个列表。（csv_character）')
			else:
				if len(list_can_complete_focus)>0:
					many_focus_many_if = ''



					for focus_id in list_can_complete_focus:
						series_focus=Df_focus.loc[Df_focus['ID']==focus_id].iloc[0, :]

						limit=''
						# check about "aviliable" in focus 
						if type(series_focus['available'])==str:
							limit+=series_focus['available']
						else:
							pass
						limit+='\n'
						
						# prerequisite focus check
						data_prerequisite = series_focus['prerequisite']
						text_prerequisite=''
						if not pandas.isnull(data_prerequisite):
							list_prerequisite = load_json(series_focus, 'prerequisite', '', "csv_focus", focus_id) 
							if type(list_prerequisite)!=list:# error: wrong data type
								print('Read focus_csv row {0} columns {1}failed, require list, got wrong data type. 读取focus_csv行{0}列{1}失败。这里应该输入一个list'.format(str(series_focus['ID'],'prerequisite')))
							else:
								if get_list_depth(list_prerequisite)!=2:# error: wrong depth
									print('Read focus_csv row {0} columns {1}failed, require depth = 2 list, got list with wrong depth. 读取focus_csv行{0}列{1}失败。这里应该输入一个2层的list'.format(str(series_focus['ID'],'prerequisite')))
								else:
									for row in list_prerequisite:
										text_row='OR = {'
										for focus in row:
											text_row+=' has_completed_focus = '+focus+'\n			'
										text_row+='}\n			'
										text_prerequisite+=text_row
						
						limit+=text_prerequisite
						# mutually_exclusive focus check
						data_mutually_exclusive = series_focus['mutually_exclusive']
						text_mutually_exclusive=''
						# has_start_focus_control_event_flag
						has_start_focus_control_event_flag=character_KEY+'_has_start_focus_control_event_flag'
						if not pandas.isnull(data_mutually_exclusive):
							list_mutually_exclusive=load_json(series_focus, 'mutually_exclusive', '', "csv_focus", focus_id) 
							if type(list_mutually_exclusive)!=list:# error: wrong data type
								print('Read focus_csv row {0} columns {1}failed, require list, got wrong data type. 读取focus_csv行{0}列{1}失败。这里应该输入一个list'.format(str(series_focus['ID'],'mutually_exclusive')))
							else:
								if get_list_depth(list_mutually_exclusive)!=1:# error: wrong depth
									print('Read focus_csv row {0} columns {1}failed, require depth = 1 list, got list with wrong depth. 读取focus_csv行{0}列{1}失败。这里应该输入一个2层的list'.format(str(series_focus['ID'],'mutually_exclusive')))
								else:
									text_row=''
									for focus in list_mutually_exclusive:
										text_row+='NOT = {\n		 has_completed_focus = '+focus +'\n		has_shine_effect_on_focus = '+focus +'\n		}\n		'
									text_mutually_exclusive+=text_row+'\n        '
						limit+=text_mutually_exclusive

						focus_in_progress_flag = project_code+"_"+focus_id+"_in_progress_flag"
						costX7_add_1= str(int(series_focus['cost'])*7+1)
						who_is_doing_variable=project_code+"_"+focus_id+"_doing_focus_advisor"
						is_on_the_seat_trigger='{0}_{1}_is_on_the_seat'.format(project_code, character_KEY)
						#### event_choose
						if series_focus['event_control_type']=="event_choose":
							# event namespace
							event_control_type_event_namespace_one_character+=['add_namespace = {}\n'.format(
								one_focus_event_choose_event_namespace.format(project_code=project_code,character_key=character_KEY,focus_id=focus_id))]
							#options
							options=''
							clr_waiting_flags = ''
							set_waiting_flags = ''
							for choose_focus_id in [focus_id]+list_mutually_exclusive:
								set_waiting_flags += 'set_country_flag = {project_code}_{focus_id}_waiting\n'.format(
									project_code=project_code,
									focus_id=choose_focus_id
								)
								clr_waiting_flags += 'clr_country_flag = {project_code}_{focus_id}_waiting\n'.format(
									project_code=project_code,
									focus_id=choose_focus_id
								)
								options+=choose_focus_event_option.format(
									project_code=project_code,
									character_id=character_id,
									character_key=character_KEY,
									focus_id=focus_id,
									limit=limit,
									choose_focus_id=choose_focus_id,
									in_progress_flag=project_code+"_"+choose_focus_id+"_in_progress_flag",
									costX7_add_1=costX7_add_1,
									who_is_doing_variable=project_code+"_"+choose_focus_id+"_doing_focus_advisor",
									circle_event_pending_flag=circle_event_pending_flag,
									clr_waiting_flags=clr_waiting_flags
								)
							# event_control_type_event
							event_control_type_event_one_character+=one_focus_event_choose_event.format(
									project_code=project_code,
									character_key=character_KEY,
									focus_id=focus_id,
									is_on_the_seat_trigger=is_on_the_seat_trigger,
									circle_event_pending_flag=circle_event_pending_flag,
									options=options,
									set_waiting_flags=set_waiting_flags
							)

							# oneif
							many_focus_many_if+=one_focus_start_one_if_event_control_type.format(
								limit=limit,
								focus_id=focus_id,
								project_code=project_code,
								event_id='{project_code}_{character_key}_choose.{focus_id}.01'.format(
									character_key=character_KEY,
									project_code=project_code,
									focus_id=focus_id
								)
							)

						#### event_redline
						elif series_focus['event_control_type']=="event_redline":
							# event namespace
							event_control_type_event_namespace_one_character+=['add_namespace = {}\n'.format(
								one_focus_event_redline_event_namespace.format(
									project_code=project_code,
									character_key=character_KEY,
									focus_id=focus_id
									)
								)
							]
							# event_control_type_event_options
								# start option
							redline_event_start_focus_option_temp=redline_event_start_focus_option.format(
								project_code=project_code,
								character_id=character_id,
								character_key=character_KEY,
								focus_id=focus_id,
								in_progress_flag=focus_in_progress_flag,
								circle_event_pending_flag=circle_event_pending_flag,
								costX7_add_1=costX7_add_1,
								who_is_doing_variable=who_is_doing_variable

							)
								# stop option
							redline_event_stop_focus_option_temp=redline_event_stop_focus_option.format(
								project_code=project_code,
								character_id=character_id,
								character_key=character_KEY,
								focus_id=focus_id,
								circle_event_pending_flag=circle_event_pending_flag,
								days_to_start_focus=days_to_start_focus,
								days_to_start_focus_random_days=days_to_start_focus_random_days

							)

							options= redline_event_start_focus_option_temp+redline_event_stop_focus_option_temp
							# event_control_type_event
							event_control_type_event_one_character+=one_focus_event_redline_event.format(
									project_code=project_code,
									character_key=character_KEY,
									character_id=character_id,
									focus_id=focus_id,
									is_on_the_seat_trigger=is_on_the_seat_trigger,
									circle_event_pending_flag=circle_event_pending_flag,
									options=options
							)
							# one_if
							many_focus_many_if+=one_focus_start_one_if_event_control_type.format(
								limit=limit,
								focus_id=focus_id,
								project_code=project_code,
								event_id="{project_code}_{character_key}_redline.{focus_id}.01".format(
									project_code=project_code,
									character_key=character_KEY,
									focus_id=focus_id
									)
							)

						#### event_remind
						elif series_focus['event_control_type']=="event_remind":
							# event namespace
							event_control_type_event_namespace_one_character+=[
								'add_namespace = {}\n'.format(
									one_focus_event_remind_event_namespace.format(
										project_code=project_code,character_key=character_KEY
									)
								)
							]
							# event_control_type_event
							event_control_type_event_one_character+=one_focus_event_remind_event.format(
									project_code=project_code,
									character_key=character_KEY,
									character_id=character_id,
									focus_id=focus_id,
									is_on_the_seat_trigger=is_on_the_seat_trigger,
									in_progress_flag=focus_in_progress_flag,
									costX7_add_1=costX7_add_1,
									who_is_doing_variable=who_is_doing_variable
							)
							# one_if
							many_focus_many_if+=one_focus_start_one_if_event_control_type.format(
								limit=limit,
								focus_id=focus_id,
								project_code=project_code,
								event_id="{project_code}_{character_key}_remind.{focus_id}".format(
									project_code=project_code,
									character_key=character_KEY,
									focus_id=focus_id
								)
							)
						
						#### do_nothing
						else:
							# one_if
							many_focus_many_if+=one_focus_start_one_if_no_control.format(
								limit=limit,
								has_start_focus_control_event_flag=has_start_focus_control_event_flag,
								character_key=character_KEY,
								character_id=character_id,
								focus_id=focus_id,
								in_progress_flag=focus_in_progress_flag,
								costX7_add_1=costX7_add_1,
								who_is_doing_variable=who_is_doing_variable,
								days_to_start_focus=days_to_start_focus,
								days_to_start_focus_random_days=days_to_start_focus_random_days,
								project_code=project_code
							)

					many_focus_many_if=many_focus_many_if.replace('else_if','if',1)
					one_start_event=start_event.format( 
							character_key=character_KEY,
							focus_in_progress_flag=focus_in_progress_flag,
							is_on_the_seat_trigger='{0}_{1}_is_on_the_seat'.format(project_code, character_KEY),
							many_focus_many_if=many_focus_many_if,
							all_completed_else=all_completed_else.format(
								all_focus_completed_trigger=project_code+'_'+character_KEY+'_all_focus_completed',
								character_key=character_KEY,
								),
							no_focus_else=one_focus_start_no_focus_else.format(
								character_key=character_KEY,
								days_to_start_focus=days_to_start_focus,
								days_to_start_focus_random_days=days_to_start_focus_random_days,
								has_start_focus_control_event_flag=has_start_focus_control_event_flag
							),
							has_start_focus_control_event_flag=has_start_focus_control_event_flag
						)
					# add
					start_focus_event_all_events+=one_start_event

					#event
					event_control_type_event_all+=event_control_type_event_one_character
					
					#namespace
					event_control_type_event_namespace_all+="".join(list(set(event_control_type_event_namespace_one_character)))

	# main start event txt
	start_focus_event_alltext=start_focus_event_all_namespace+'\n'+start_focus_event_all_events

	# event_control_type focus event txt
	event_control_type_focus_events_alltext=event_control_type_event_namespace_all+'\n'+event_control_type_event_all

	# write main start event txt
	with open(os.path.join(dir_output,'events',project_code+'_start_focus_event.txt'),'w',encoding='utf_8', newline='\n')as f:
		f.write(start_focus_event_alltext)

	# write event_control_type focus event txt
	with open(os.path.join(dir_output,'events',project_code+'_event_control_type_focus_event.txt'),'w',encoding='utf_8', newline='\n')as f:
		f.write(event_control_type_focus_events_alltext)
start_focus_event(csv_characters,csv_focus,dir_output)


In [49]:

# complete_focus_event
def complete_focus_event(csv_characters,csv_focus,dir_output):
	Df_characters=pandas.read_csv(csv_characters)
	Df_focus=pandas.read_csv(csv_focus)
	#mkdir
	try:
		os.mkdir(dir_output)
	except:
		pass
	try:
		os.mkdir(os.path.join(dir_output,'events'))
	except:
		pass

	complete_focus_event_all_namespace=''
	complete_focus_event_all_events=''

	for row in Df_characters.iterrows():
		s_character=row[1]
		character_KEY=s_character['KEY']
		character_id=s_character['ID']
		complete_focus_order_data = s_character['complete_focus_order']
		has_start_focus_control_event_flag=character_KEY+'_has_start_focus_control_event_flag'

		# namespace
		namespace=add_namespace.format(character_key=character_KEY)
		complete_focus_event_all_namespace+=namespace

		#events
		if type(complete_focus_order_data)==str:
			try:
				list_can_complete_focus=json.loads(complete_focus_order_data)
			except Exception as e :
				print('Read character_csv {0}\'s {1} data failed. 读取character_csv中{0}的{1}数据失败。'.format(str(character_KEY,'complete_focus_order')))
				print()
				raise e

			if not type(list_can_complete_focus)==list:
				print(character_KEY,': Read complete_focus_order failed, require a list. 读取"complete_focus_order"数据失败，请输入一个列表。（csv_character）')
			else:
				if len(list_can_complete_focus)>0:
					many_focus_many_if = ''
					for focus_id in list_can_complete_focus:
						series_focus=Df_focus.loc[Df_focus['ID']==focus_id].iloc[0, :]

						limit=''
						# check about "aviliable" in focus 
						if type(series_focus['available'])==str:
							limit+=series_focus['available']
						else:
							pass
						limit+='\n'
						
						# prerequisite focus check
						data_prerequisite = series_focus['prerequisite']
						text_prerequisite=''
						if pandas.isnull(data_prerequisite):
							pass
						else:
							list_prerequisite = load_json(series_focus, 'prerequisite', '', "csv_focus", focus_id) 
							if type(list_prerequisite)!=list:# error: wrong data type
								print('Read focus_csv row {0} columns {1}failed, require list, got wrong data type. 读取focus_csv行{0}列{1}失败。这里应该输入一个list'.format(str(series_focus['ID'],'prerequisite')))
							else:
								if get_list_depth(list_prerequisite)!=2:# error: wrong depth
									print('Read focus_csv row {0} columns {1}failed, require depth = 2 list, got list with wrong depth. 读取focus_csv行{0}列{1}失败。这里应该输入一个2层的list'.format(str(series_focus['ID'],'prerequisite')))
								else:
									for row in list_prerequisite:
										text_row='OR = {'
										for focus in row:
											text_row+=' has_completed_focus = '+focus+'\n			'
										text_row+='}\n			'
										text_prerequisite+=text_row
						limit+=text_prerequisite
						# mutually_exclusive focus check
						data_mutually_exclusive = series_focus['mutually_exclusive']
						text_mutually_exclusive=''
						# has_complete_focus_control_event_flag
						has_complete_focus_control_event_flag=character_KEY+'_has_complete_focus_control_event_flag'
						if not pandas.isnull(data_mutually_exclusive):
							list_mutually_exclusive = load_json(series_focus, 'mutually_exclusive', '', "csv_focus", focus_id) 
							if type(list_mutually_exclusive)!=list:# error: wrong data type
								print('Read focus_csv row {0} columns {1}failed, require list, got wrong data type. 读取focus_csv行{0}列{1}失败。这里应该输入一个list'.format(str(series_focus['ID'],'mutually_exclusive')))
							else:
								if get_list_depth(list_mutually_exclusive)!=1:# error: wrong depth
									print('Read focus_csv row {0} columns {1}failed, require depth = 1 list, got list with wrong depth. 读取focus_csv行{0}列{1}失败。这里应该输入一个2层的list'.format(str(series_focus['ID'],'mutually_exclusive')))
								else:
									text_row=''
									for focus in list_mutually_exclusive:
										text_row+='NOT = { has_completed_focus = '+focus +'\n		has_shine_effect_on_focus = '+focus +'}\n		'
									text_mutually_exclusive+=text_row+'\n        '
						limit+=text_mutually_exclusive

						focus_in_progress_flag = project_code+"_"+focus_id+"_in_progress_flag"

						many_focus_many_if+=one_focus_complete_one_if.format(
                            focus_id=focus_id,
                            character_key=character_KEY,
							who_is_doing_variable=project_code+"_"+focus_id+"_doing_focus_advisor",
							character_id=character_id,
							limit=limit,
							in_progress_flag=focus_in_progress_flag,
							has_start_focus_control_event_flag=has_start_focus_control_event_flag,
							days_to_start_focus=days_to_start_focus,
							days_to_start_focus_random_days=days_to_start_focus_random_days

						)

					many_focus_many_if=many_focus_many_if.replace('else_if','if',1)
					one_complete_event=complete_event.format( 
							character_key=character_KEY,
                            focus_in_progress_flag=focus_in_progress_flag,
							is_on_the_seat_trigger='{0}_{1}_is_on_the_seat'.format(project_code, character_KEY),
                            many_focus_many_if=many_focus_many_if,
							no_focus_else=one_focus_complete_no_focus_else.format(
								character_key=character_KEY,
								in_progress_flag=focus_in_progress_flag,
								has_start_focus_control_event_flag=has_start_focus_control_event_flag,
								days_to_start_focus=days_to_start_focus,
								days_to_start_focus_random_days=days_to_start_focus_random_days)
						)
					complete_focus_event_all_events+=one_complete_event
	
	complete_focus_event_alltext=complete_focus_event_all_namespace+'\n'+complete_focus_event_all_events
	# write txt
	with open(os.path.join(dir_output,'events',project_code+'_complete_focus_event.txt'),'w',encoding='utf_8', newline='\n')as f:
		f.write(complete_focus_event_alltext)
complete_focus_event(csv_characters,csv_focus,dir_output)
			